In [1]:
import sqlparse

from cast.cache import PostData

from django.test.client import RequestFactory
from django.db import connection, reset_queries

# Measure PostData Cache Performance

Find out which pages generate queries.

In [2]:
def show_queries(queries):
    for query in queries:
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        print(formatted_sql)
        

def blocker(*args):
    raise Exception("No database access allowed here.")

In [3]:
# blog_slug = "ephes_blog"
blog_slug = "das_claas_und_nora_blog"
blog = Blog.objects.get(slug=blog_slug)
request_factory = RequestFactory()

# Just measure for a list of Posts

In [4]:
%%time
reset_queries()
post_queryset = Post.objects.live().descendant_of(blog).order_by("-visible_date")
post_data = PostData.create_from_post_queryset(
    request=request_factory.get("/"),
    blog=blog,
    post_queryset=post_queryset,
    template_base_dir="bootstrap4",
)
print(len(connection.queries))

305
CPU times: user 667 ms, sys: 27 ms, total: 694 ms
Wall time: 755 ms


In [5]:
post = Post.objects.get(pk=501)

In [6]:
%%time
request = request_factory.get("/foobar/")
posts = list(post_queryset)
reset_queries()
# request = request_factory.get(post_data.page_url_by_id[post.pk])
for post in posts:
    # print("post pk: ", post.pk)
    with connection.execute_wrapper(blocker):
        post._local_template_name = "post_body.html"
        response = post.serve(request, post_data=post_data).render()

CPU times: user 975 ms, sys: 85.8 ms, total: 1.06 s
Wall time: 1.07 s


In [7]:
print(len(posts))

312


In [8]:
print(len(connection.queries))

0


In [9]:
show_queries(connection.queries)

# Feed Performance

In [10]:
from cast.feeds import LatestEntriesFeed

In [11]:
%%time
reset_queries()
post_queryset = Post.objects.live().descendant_of(blog).order_by("-visible_date")
post_data = PostData.create_from_post_queryset(
    request=request_factory.get("/"),
    blog=blog,
    post_queryset=post_queryset,
    template_base_dir="bootstrap4",
)
print(len(connection.queries))

11
CPU times: user 480 ms, sys: 10.8 ms, total: 491 ms
Wall time: 532 ms


In [12]:
# call this cell once without blocker to populate SITE_CACHE
response = LatestEntriesFeed(post_data=post_data)(request, slug=blog_slug)

In [13]:
%%time
request = request_factory.get("/foobar/")
reset_queries()

with connection.execute_wrapper(blocker):
    response = LatestEntriesFeed(post_data=post_data)(request, slug=blog_slug)

CPU times: user 1.01 s, sys: 85.9 ms, total: 1.1 s
Wall time: 1.11 s


In [14]:
print(len(connection.queries))

0


In [15]:
show_queries(connection.queries)

# Render A Blog Index Page

In [16]:
from cast.models import Blog

In [17]:
%%time
reset_queries()
post_queryset = blog.unfiltered_published_posts
post_data = PostData.create_from_post_queryset(
    request=request_factory.get("/"),
    blog=blog,
    post_queryset=post_queryset,
    template_base_dir="bootstrap4",
)
print(len(connection.queries))

11
CPU times: user 456 ms, sys: 7.97 ms, total: 464 ms
Wall time: 480 ms


In [27]:
%%time
# just like in page.py::Blog but externally
reset_queries()
request = request_factory.get("/foobar/")
get_params = request.GET.copy()
context = {
    "page": blog,
    "blog": blog,
    "request": request,
}
context["filterset"] = filterset = blog.get_filterset(get_params)
context["parameters"] = blog.get_other_get_params(get_params)
context = blog.paginate_queryset(context, blog.get_published_posts(filterset.qs), get_params)
context["posts"] = context["object_list"]
context["has_selectable_themes"] = True
context["template_base_dir"] = blog.get_template_base_dir(request)
context["use_audio_player"] = any([post.has_audio for post in context["posts"]])
context["theme_form"] = blog.get_theme_form(request)
print(len(connection.queries))

12
CPU times: user 33.7 ms, sys: 10.7 ms, total: 44.4 ms
Wall time: 50.5 ms


In [29]:
blog.get_url()

'/blogs/das_claas_und_nora_blog/'

In [28]:
print(context)

{'page': <Blog: Claas und Nora sagen Hallo>, 'blog': <Blog: Claas und Nora sagen Hallo>, 'request': <WSGIRequest: GET '/foobar/'>, 'filterset': <cast.filters.PostFilterset object at 0x14081bda0>, 'parameters': '', 'paginator': <django.core.paginator.Paginator object at 0x1405c8680>, 'page_obj': <Page 1 of 63>, 'is_paginated': True, 'object_list': [<Post: März 2024-03-11>, <Post: März 2024-03-04>, <Post: Februar 2024-02-26>, <Post: Februar 2024-02-19>, <Post: Februar 2024-02-12>], 'renditions_for_posts': {2397: [<Rendition: Rendition object (80971)>, <Rendition: Rendition object (80975)>, <Rendition: Rendition object (80976)>, <Rendition: Rendition object (80983)>, <Rendition: Rendition object (80984)>, <Rendition: Rendition object (80985)>, <Rendition: Rendition object (80986)>, <Rendition: Rendition object (80987)>, <Rendition: Rendition object (80988)>, <Rendition: Rendition object (80989)>, <Rendition: Rendition object (80990)>, <Rendition: Rendition object (80991)>, <Rendition: Ren

In [19]:
%%time
request = request_factory.get("/foobar")
request.htmx = False
reset_queries()

# with connection.execute_wrapper(blocker):
response = blog.serve(request)

get_params:  <QueryDict: {}>
context:  {'page': <Blog: Claas und Nora sagen Hallo>, 'self': <Blog: Claas und Nora sagen Hallo>, 'request': <WSGIRequest: GET '/foobar'>}
CPU times: user 40.7 ms, sys: 10.9 ms, total: 51.6 ms
Wall time: 60.1 ms


In [53]:
print(len(connection.queries))

12


In [52]:
show_queries(connection.queries)

SELECT "cast_postcategory"."slug",
       "cast_postcategory"."name",
       COUNT("cast_post_categories"."post_id") FILTER (
                                                       WHERE "cast_post_categories"."post_id" IN
                                                           (SELECT U0."page_ptr_id"
                                                            FROM "cast_post" U0
                                                            INNER JOIN "wagtailcore_page" U1 ON (U0."page_ptr_id" = U1."id")
                                                            WHERE (U1."live"
                                                                   AND U1."path"::text LIKE '000100010001%'
                                                                   AND U1."depth" >= 3
                                                                   AND NOT (U0."page_ptr_id" = 3)))) AS "num_posts"
FROM "cast_postcategory"
LEFT OUTER JOIN "cast_post_categories" ON ("cast_postcategory"."id" = "cast